In [1]:
from operator import itemgetter

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

model = ChatOllama(model="mistral")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

" La ciudad de donde nació Barack Obama es Honolulu, Hawái, Estados Unidos. Sin embargo, pasó la mayor parte de su infancia en Jakarta, Indonesia, y más tarde se mudó a Chicago, Illinois, donde comenzó su carrera política. Así pues, aunque no creció en una ciudad asociada comúnmente con él, la ciudad de la que es originalmente oriundo es Honolulu.\n\nTherefore, the city that Barack Obama was born in is Honolulu, Hawaii, United States. However, he spent most of his childhood in Jakarta, Indonesia, and later moved to Chicago, Illinois, where his political career began. So while he was not raised in a city commonly associated with him, the city from which he originally comes from is Honolulu.\n\nI hope this helps clarify any confusion! Let me know if you have any other questions or if there's anything else I can help you with. 😊"

In [2]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser

question_generator =  color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4


In [3]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of  Mango (This fruit is often associated with the color orange or yellow in various shades.) and the flag of  The flag of Palestine has a prominent use of the color green, but it also includes red and white. Therefore, based on your instruction asking for a "warm color," I would suggest the color "Tangerine" as an option, even though it is not exactly present in the Palestinian flag. Other flags with warm colors as their primary hue include the flags of Nigeria (Green-White-Green with a yellow sun emblem), Libya (Red-Black-Green), and Turkmenistan (Turkmen Banner of Star with red, green, and blue). Ultimately, the choice depends on your specific requirement or preference.?')])

In [4]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content=' An apple can come in various shades of red, among other colors such as green or yellow, depending on its ripeness and variety. The flag of China has three main bands of color: red, which is located at the top and bottom edges, and a larger band in the middle with a yellow sun and stars on it. So, while apples can be red, the color red also significantly features on the Chinese flag.')

## Branching and Merging